In [ ]:
from typing import Any, Final
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import numpy as np
import matplotlib.pyplot as plot
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg") #type: ignore

## Setup and organize data

In [ ]:
import seaborn as sns
import torch

# convert from pandas to tensor
iris = sns.load_dataset("iris")
data = torch.tensor(iris[iris.columns[0:4]].values).float() # type: ignore

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
versicolors = iris.species == "versicolor"
labels[versicolors] = 1
labels[iris.species == "virginica"] = 2
labels

In [ ]:
class AnnMod(nn.Module):
    def __init__(
        self,
        inputs: int,
        in_outs: int,
        outputs: int
    ):
        super().__init__()  # type: ignore
        self.inputs = inputs
        self.in_outs = in_outs
        self.outputs = outputs
        self._num_layers: int = 0
        self.layers = nn.ModuleDict()
        self.output_name: Final[str] = "output"
        self._hidden: list[str] = []

        self.layers["input"] = nn.Linear(self.inputs, self.in_outs)

    @property
    def num_layers(self):
        return self._num_layers
    
    @num_layers.setter
    def num_layers(self, _val: int):
        print("Can not set")

    def hidden_layer(self, name: str, mod: nn.Module):
        self._hidden.append(name)
        self.layers[name] = mod
        self._num_layers += 1
        return self
    
    def output_layer(self, mod: nn.Module):
        self.layers[self.output_name] = mod
        self._num_layers += 1
        return self
    
    def forward(self, inputs: Any):
        # input ayer
        x: Tensor = self.layers["input"](inputs)

        # hidden layers
        # TODO: associate non-linear activation with each hidden layer
        for hidden in self._hidden:
            lin: Tensor = self.layers[hidden](x)
            x = F.relu(lin)
        
        # Output layer
        x = self.layers[self.output_name](x)


In [ ]:
am = AnnMod(4, 12, 3)
for i in range(4):
    am.hidden_layer(f"hidden{i}", nn.Linear(am.in_outs, am.in_outs))
am.output_layer(nn.Linear(am.in_outs, am.outputs))